In [1]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv('../data/authors_mastodon_papers_07-31-2023.tsv', sep='\t')

In [14]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

In [35]:
data.columns

Index(['id', 'doi', 'doi_registration_agency', 'display_name', 'title',
       'publication_year', 'publication_date', 'language', 'ids',
       'primary_location', 'best_oa_location', 'type', 'open_access',
       'authorships', 'corresponding_author_ids',
       'corresponding_institution_ids', 'cited_by_count', 'summary_stats',
       'biblio', 'is_retracted', 'is_paratext', 'concepts', 'mesh',
       'locations_count', 'locations', 'referenced_works', 'grants',
       'apc_payment', 'related_works', 'abstract_inverted_index',
       'counts_by_year', 'cited_by_api_url', 'updated_date', 'created_date'],
      dtype='object')

In [37]:
data['cited_by_count'].head(30)

0      0
1      0
2      0
3     53
4      0
5      1
6      1
7      0
8      0
9      3
10     0
11     0
12     0
13     1
14    15
15     0
16     0
17     0
18     6
19     0
20     1
21     0
22     0
23     0
24     0
25     0
26     2
27     0
28     1
29     0
Name: cited_by_count, dtype: int64

In [49]:
def get_cited_by(url):
    r = session.get(url, params={'mailto':'carolmedeirosbrito@gmail.com'})
    return r.json()

for idx, row in data[(data['publication_year'] < 2010) & (data['cited_by_count'] > 0)][:10].iterrows():
    print(row['title'])
    print(row['doi'])
    print(row['cited_by_count'])
    print(get_cited_by(row['cited_by_api_url']))
    print()
# for idx, row in (data[data['cited_by_count'] > 0]).iterrows():
#     print(get_cited_by(row['cited_by_api_url'])['meta']['count'])

The new cryptographic hash function RIPEMD-160
nan
1
{'meta': {'count': 0, 'db_response_time_ms': 88, 'page': 1, 'per_page': 25}, 'results': [], 'group_by': []}

A visual design for collaborative work: columns for commenting and annotation
https://doi.org/10.1109/hicss.1991.184207
15
{'meta': {'count': 14, 'db_response_time_ms': 49, 'page': 1, 'per_page': 25}, 'results': [{'id': 'https://openalex.org/W1973304785', 'doi': 'https://doi.org/10.1145/143457.143473', 'title': 'Flexible Diff-ing in a collaborative writing system', 'display_name': 'Flexible Diff-ing in a collaborative writing system', 'publication_year': 1992, 'publication_date': '1992-01-01', 'ids': {'openalex': 'https://openalex.org/W1973304785', 'doi': 'https://doi.org/10.1145/143457.143473', 'mag': '1973304785'}, 'language': 'en', 'primary_location': {'is_oa': True, 'landing_page_url': 'https://doi.org/10.1145/143457.143473', 'pdf_url': 'https://dl.acm.org/doi/pdf/10.1145/143457.143473', 'source': None, 'license': None, 'v

In [31]:
len(data)

111262